In [2]:
#import ipywidget
from ipywidgets      import widgets,interact,interactive
from IPython.display import display
import pandas as pd
import os
from pathlib import Path
import plotly.graph_objects as go
from PIL import Image

'''
csv data format

    "game starttime",
    "game endtime",
    "gameId",                counter
    "offense_team_id",       when event=goal|shot     team offenser 
    "offense_team_name",     when event=goal|shot     team name
    "offense_team_tricode",  when event=goal|shot     team tricode
    "goal",                  goal = 1 | shot= 0
    "x_coords",              
    "y_coords",
    "goalie_id",             goalkeeper 
    "goalie_name",           goalkeeper
    "shooter_id",
    "shooter_name",
    "shot type",
    "empty net",
    "strength_shorthand",
    "strength_even",
    "strength_powerplay",
    
    calcule el total estadistico de lugares de disparos para toda la liga 
    para calcular la /media de disparos por hora = XX/

    agrupe los disparos por equipo, y use la /media de la liga por hora=XX/ 
    para calcular el exceso de disparos por hora.
    
    puede elegir representarlo como diferencia numerica en goles entre equipos o en porcentaje
    
    
    Compute aggregate statistics of shot locations across the entire league 
    to compute league average shot rate per hour. 
    You can make a few simplifying assumptions:
    You can assume all shots are even strength; this means you can simply aggregate over all shots rather than having to figure out whether a shot was an even strength shot or not (recall Q 4.2).
    You can assume each game lasts 60 minutes.
    Group shots by team, and use the league average shot rate per hour computed above 
    to compute the excess shot rate per hour. 
    You can choose to represent this as either a raw difference in goals between the teams, or a percentage. 

'''
full_path = os.getcwd()
root_path = str(Path(full_path).parents[0])

#define initial parameters
im      = Image.open(r'{r}/figures/nhl_rink.png'.format(r=root_path)) 
im2     = Image.open(r'{r}/figures/nhl_rink_top.png'.format(r=root_path)) 
seasons = ['20162017','20172018','20182019','20192020', '20202021']
teams   = ['NOP','NJD','NYI','NYR','PHI','PIT','BOS','BUF','MTL','OTT','TOR','FLA','WSH','CHI','DET','NSH','STL','CGY','COL','EDM','VAN','ANA','DAL','LAK','SJS','CBJ','MIN','WPG','ARI','VGK','SEA']

print('csv on ',root_path+"/ift6758/data/tabular") 


csv on  /Users/olivercordoba/Documents/github/IFT6758/ift6758/data/tabular


In [3]:
'''
 based on season and team, draw all shooting point on the map
 
'''
def vgraph(season,team):
    
    # Create figure
    fig = go.Figure()
    
    print('season: {s} team: {t} '.format(s=season,t=team))
    
    #load data for selected season / team
    csv_path  = root_path+'/ift6758/data/tabular/'+season+'.csv'
    df = pd.read_csv(csv_path)
    df2 =df[df['offense_team_tricode'] == team]
    
    # Add traces
    fig.add_trace(
        go.Scatter(
            x=df2.loc[:, 'x_coords'],
            y=df2.loc[:, 'y_coords'],
            mode="markers",
            marker=dict(color="DarkOrange"),
        )
    )

   
    # Add background image
    fig.add_layout_image(
        dict(
            source=im,
            xref="x",
            yref="y",
            x=-100,   # dims in feet
            y=42.5,
            sizex=200,
            sizey=85,
            sizing="stretch",
            opacity=0.8,
            layer="below"
        )
    )
    
    
    #run view
    fig.update_layout(
        template="plotly_white"
    )
    fig.show()
    
    
w = interactive(vgraph,  season=widgets.Dropdown(options=seasons, value='20162017', description='Season',),
                         team  =widgets.Dropdown(options=teams,   value='MTL',      description='Team'),
                       )


display(w)
    

interactive(children=(Dropdown(description='Season', options=('20162017', '20172018', '20182019', '20192020', …

In [116]:
'''
 based on season and team, draw all shooting point on the map
 
'''
def vgraph(season,team):
    
    # Create figure
    fig = go.Figure()
    
    print('season: {s} team: {t} '.format(s=season,t=team))
    
    #load data for selected season / team
    csv_path  = root_path+'/ift6758/data/tabular/'+season+'.csv'
    df = pd.read_csv(csv_path)
    
    
    
    
    #league shoot count , create unique key
    df['team_game'] = df.apply(lambda row: str(row['game starttime'])+str(row['offense_team_tricode']), axis=1)
    l_shoot_count  =  df.shape[0] 
    # equals a shoot count / season_games , asumming games are 60 mins
    l_season_games = df['team_game'].unique().shape[0]
    ashotr = l_shoot_count/l_season_games;  #around 30
    #print('---league----')
    #print('--season games counter = ',l_season_games)
    #print('--season games shoots  = ',l_shoot_count)
    #print('--average shoot rate per hour = ',ashotr) 
  
    # create new column abs(x) will be the new Y 
    df['y_abs'] = df.apply(lambda row: (abs(row.x_coords)-89.0)*(-1.0), axis = 1)
    
    # create a new dataframe for current team
    df2 =df[df['offense_team_tricode'] == team].copy()
    
    
    #compute shots on current game
    df3 = df2.groupby(['game starttime'])['game starttime'].count().to_frame()
    
    # display('count shoot by date by current team ')
    #print('games counter df3-------')
    #print(df3)
    
    # total shots on current game is equal at number of records on this date for this team 
    # index value is the game starttime 
    # df3.loc[row['game starttime'],:]['game starttime']
    
    team_season_games = df2['game starttime'].unique().shape[0]
    
    # get the average shot rate all leagues all teams on each feet of the floor
    # we will map a dataframe of 80 x 85 records
    # print(df2.head())
    
    #tile_sz = 1 
    #x_range = list(range(-43,43,1))
    #y_range = list(range(-11,85,1))
    tile_sz = 5 
    x_range = list(range(-50,50,tile_sz))
    y_range = list(range(-10,80,tile_sz))
    
    #df_lmean= pd.DataFrame(columns=['x', 'y', 'all_shots', 'team_game_shoots', 'mean' , 'team_shoots'])

    for xval in x_range:
        for yval in y_range:
            # all shoot inside the current tile
            tile_shots = df.loc[(df['y_coords']>=xval)           & 
                                (df['y_coords']< (xval+tile_sz)) & 
                                (df['y_abs']   >=yval)           &
                                (df['y_abs']   < (yval+tile_sz)) ]
            tile_count = tile_shots.shape[0]
            tile_mean_per_game  = tile_count/l_season_games
            # team shoot inside current tile
            teamtile_shoots = df2.loc[(df2['y_coords']>=xval)           & 
                                      (df2['y_coords']< (xval+tile_sz)) & 
                                      (df2['y_abs']   >=yval)           &
                                      (df2['y_abs']   < (yval+tile_sz)) ]
            tg_shoots = teamtile_shoots.shape[0]
            tile_mean_per_game_per_team = tg_shoots/team_season_games
            #find all records that match this coordinates to include the value 
            for idx in teamtile_shoots.index:
                df2.loc[idx,'excess_shot_rate'] = tile_mean_per_game_per_team -  tile_mean_per_game 
                
            '''
            df_lmean = pd.concat([df_lmean, 
                pd.DataFrame([{'x':xval, 
                               'y':yval, 
                               'all_shots':tile_count,
                               'team_game_shoots':tg_shoots, 
                               'mean':tile_mean_per_game,
                               'team_shoots':tile_mean_per_game_per_team
                              }])
                        ])
            ''' 
            
    #print('-----medias----')                
    #print(df2)     
    
    #df2[['y_coords','y_abs']].groupby('iso_yrwk').agg(['mean','count'])
    
    # for each value x,y we have to find the values of the league average and compare
    # against the value of the team, this is the excess_shot_rate
    # df2['excess_shot_rate']= df2.apply(lambda row:  df3.loc[row['game starttime'],:]['game starttime'] - ashotr,axis = 1 )
    
    
    #print(df2)
    
    #team average shot rate per hour
    #shoot_count  =  df2.shape[0] 
    # equals a shoot count / season_games , asumming games are 60 mins
    #season_games = df2['game starttime'].unique().shape[0]
    #print('----team----')
    #print('--season games counter = ',season_games)
    #print('--season games shoots  = ',shoot_count)
    #print('--average shoot rate per hour = ',shoot_count/season_games)
    
    
    
    # Add traces
    '''
    fig.add_trace(
        go.Scatter(
            x=df2.loc[:, 'y_coords'],
            y=df2.loc[:, 'y_abs'],
            mode="markers",
            marker=dict(color="DarkOrange"),
        )
    )
    '''
    # work in progress: need to undestand how to get the league avg shot 
   
    fig.add_trace(
        go.Contour(
            z=df2.loc[:, 'excess_shot_rate'],
            x=df2.loc[:, 'y_coords'],
            y=df2.loc[:, 'y_abs'],
            opacity=0.6,
        colorscale='Viridis'
        )
    )
    '''
    
    fig.add_trace(
        go.Surface((
            x=df2.loc[:, 'y_coords'],
            y=df2.loc[:, 'y_abs'],
            z=df2.loc[:, 'excess_shot_rate'],
        colorscale='Viridis'
        )
    )
    '''
   
    # Add background image
    fig.add_layout_image(
        dict(
            source=im2,
            xref="x",
            yref="y",
            x=-42.5,   # dims in feet
            y=-11,
            sizex=85,
            sizey=100,
            sizing="stretch",
            layer="below"
        )
    )
    
    # reverse Y axis to be top-down direction
    fig.update_yaxes(
        autorange="reversed"
    )
    
    # header title 
    fig.add_annotation(x=.5, y=-22,
            text="Unblocked Shot Rate",
            showarrow=False,
            font=dict(
                family="verdana, Arial",
                size=18, 
                color="#333333",
            )
    )
    
    # axes, scales and margins
    fig.update_layout(
        showlegend=False,
        xaxis_title="Distace from cenre of rink (ft)",
        yaxis_title="Distance from goal line (ft)",
        font=dict(
            family="Courier New, monospace",
            size=12,
            color="#333333"
        ),
        width=85*8,
        height=100*8,
        autosize=True,
        margin=dict(t=50, b=0, l=50, r=0), # graph margins
        template="plotly_white"
    )

        
    # run the visualization    
    fig.show()
    
    
w = interactive(vgraph,  season=widgets.Dropdown(options=seasons, value='20162017', description='Season',),
                         team  =widgets.Dropdown(options=teams,   value='MTL',      description='Team'),
                       )


display(w)

interactive(children=(Dropdown(description='Season', options=('20162017', '20172018', '20182019', '20192020', …

In [65]:
csv_path  = root_path+'/ift6758/data/tabular/'+'20162017'+'.csv'
df = pd.read_csv(csv_path)

In [66]:
df.head()

,game starttime,game endtime,gameId,offense_team_id,offense_team_name,offense_team_tricode,goal,x_coords,y_coords,goalie_id,goalie_name,shooter_id,shooter_name,shot type,empty net,strength_shorthand,strength_even,strength_powerplay
0,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,0,10,Toronto Maple Leafs,TOR,0,-77.0,5.0,8467950,Craig Anderson,8478483,Mitchell Marner,Wrist Shot,False,NaN,NaN,NaN
1,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,1,9,Ottawa Senators,OTT,0,86.0,13.0,8475883,Frederik Andersen,8467967,Chris Kelly,Wrist Shot,False,NaN,NaN,NaN
2,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,2,9,Ottawa Senators,OTT,0,23.0,-38.0,8475883,Frederik Andersen,8476879,Cody Ceci,Wrist Shot,False,NaN,NaN,NaN
3,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,3,9,Ottawa Senators,OTT,0,33.0,-15.0,8475883,Frederik Andersen,8474578,Erik Karlsson,Slap Shot,False,NaN,NaN,NaN
4,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,4,10,Toronto Maple Leafs,TOR,0,-34.0,28.0,8467950,Craig Anderson,8475716,Martin Marincin,Wrist Shot,False,NaN,NaN,NaN
